In [1]:
# 분석 라이브러리
import pandas as pd
import numpy as np

# Open API 활용
import requests
from bs4 import BeautifulSoup 
import os, re, time
from tqdm import tqdm

# 자연어처리 라이브러리
import torchtext
from konlpy.utils import pprint
from konlpy.tag import Okt

# 사이킷런 패키지 활용 
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis.sklearn

# 경고 문구 미표시
import warnings
warnings.filterwarnings("ignore")

In [2]:
# 다운로드 받을 폴더를 준비
DATA_DIR = "./data"
os.makedirs(DATA_DIR, exist_ok=True)

# 데이터 다운로드
torchtext.utils.download_from_url(url='https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt', path=os.path.join(DATA_DIR, 'review.txt'))

# txt 파일을 판다스 데이터프레임으로 읽어오기
data = pd.read_csv('/Users/yoongyo/Study/데이터분석프로젝트/data/review.txt', sep='\t')

In [3]:
data

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [21]:
# 트위터 형태소 분석기(Okt)를 활용
okt = Okt()

In [22]:
okt

In [5]:
# 누락 데이터 제거
review_data = data['document'].dropna().values

# 세 글자 이상의 명사를 사용 (두 글자 이하의 단어는 제거)
cleaned_review_data = []
for review in tqdm(review_data):
    # # 명사 ONLY 추출
    tokens = okt.nouns(review)
    cleaned_tokens = []
    for word in tokens:
        if len(word) > 2:
            cleaned_tokens.append(word)
        else:
            pass
    cleaned_review = " ".join(cleaned_tokens)
    cleaned_review_data.append(cleaned_review)


100%|██████████| 149995/149995 [07:08<00:00, 349.91it/s]


In [6]:
cleaned_review_data

['목소리',
 '포스터',
 '',
 '교도소 이야기',
 '스파이더맨 커스틴 던스트',
 '초등학교',
 '긴장감 제대로',
 '이응경 길용우 드라마',
 '',
 '헐리우드',
 '인피니트',
 '허진호',
 '횡단보도 이범수',
 '',
 '스토리 어거지 어거지',
 '재밋음',
 '바스코 아이돌',
 '굿바이 갈수록',
 '캐스팅 용구성 버무러진',
 '',
 '',
 '',
 '바베트 바베트 이야기 핀란드',
 '',
 '',
 '짤랐을꺼',
 '',
 '카밀라 발연기',
 '재밋는뎅',
 '캐스팅',
 '엄포스 드라마',
 '쓰레기',
 '',
 '',
 '드라마',
 '',
 '키이라 나이틀리 정신장애 틱장애',
 '',
 '포스터',
 '',
 '',
 '알바생',
 '',
 '',
 '윤제문',
 '',
 '리얼리티 이민기 캐릭터 정신의학 분노조절 툭하면 갈수록 이민기',
 '마이너스',
 '',
 '근친상간 드라마 드래곤',
 '',
 '세르게이',
 '드라마',
 '',
 '',
 '',
 '',
 '일본인 상상력',
 '',
 '백봉기',
 '그대로 카리스마',
 '',
 '',
 '전개도 주인공',
 '가슴속',
 '우리나라 민간인 살인자',
 '캐릭터 에피소드 시청률',
 '김남길 연기력 몰입도 손예진',
 '',
 '노래실력 박시환',
 '',
 '아무나',
 '',
 '재밋네 달팽이',
 '',
 '로마노프 러시아',
 '',
 '',
 '한국영',
 '아햏햏 아햏햏 아햏햏',
 '리플릿',
 '',
 '',
 '우뢰매',
 '쓰레기',
 '',
 '',
 '',
 '막문위',
 '',
 '',
 '걸스데이 이혜리',
 '',
 '',
 '',
 '어내스트 셀레스틴',
 '',
 '클라라',
 '',
 '에피소드 스토리',
 '신카이 마코토',
 '캐스팅',
 '',
 '고은님',
 '스토리',
 '',
 '킬링타임',
 '크리스마스',
 '',
 '',
 '로맨스',
 '',
 '별루더',
 '',
 '시청률 드라마',
 '',

In [7]:
# TF-IDF 변환기 객체를 생성
tfid = TfidfVectorizer()

# TF-IDF 변환기에 데이터를 입력하여 변환
review_tfid = tfid.fit_transform(cleaned_review_data)

# 단어 사전 확인 (딕셔너리 형태)
vocab = tfid.vocabulary_

In [23]:
vocab

{'목소리': 4849,
 '포스터': 15807,
 '교도소': 976,
 '이야기': 11680,
 '스파이더맨': 8274,
 '커스틴': 14689,
 '던스트': 3041,
 '초등학교': 14235,
 '긴장감': 1437,
 '제대로': 13132,
 '이응경': 11749,
 '길용우': 1447,
 '드라마': 3401,
 '헐리우드': 16613,
 '인피니트': 12038,
 '허진호': 16600,
 '횡단보도': 16958,
 '이범수': 11531,
 '스토리': 8206,
 '어거지': 9551,
 '재밋음': 12635,
 '바스코': 5442,
 '아이돌': 9038,
 '굿바이': 1125,
 '갈수록': 134,
 '캐스팅': 14651,
 '용구성': 10790,
 '버무러진': 5901,
 '바베트': 5435,
 '핀란드': 16145,
 '짤랐을꺼': 13845,
 '카밀라': 14559,
 '발연기': 5710,
 '재밋는뎅': 12561,
 '엄포스': 9822,
 '쓰레기': 8785,
 '키이라': 15051,
 '나이틀리': 2132,
 '정신장애': 13026,
 '틱장애': 15465,
 '알바생': 9299,
 '윤제문': 11281,
 '리얼리티': 4130,
 '이민기': 11517,
 '캐릭터': 14642,
 '정신의학': 13025,
 '분노조절': 6419,
 '툭하면': 15344,
 '마이너스': 4282,
 '근친상간': 1310,
 '드래곤': 3420,
 '세르게이': 7545,
 '일본인': 12088,
 '상상력': 7174,
 '백봉기': 5838,
 '그대로': 1204,
 '카리스마': 14546,
 '전개도': 12799,
 '주인공': 13489,
 '가슴속': 52,
 '우리나라': 10822,
 '민간인': 5311,
 '살인자': 7108,
 '에피소드': 9959,
 '시청률': 8472,
 '김남길': 1483,
 '연기력': 10151,
 '몰입도': 4889,


In [8]:
labels = data['label'].iloc[:149995].values

In [9]:
# LDA 모델링 객체를 생성 (토픽 갯수를 2로 설정: 긍정/부정)
lda = LatentDirichletAllocation(n_components=2)  

# TF-IDF 벡터를 입력하여 모델 학습 
lda.fit(review_tfid)

LatentDirichletAllocation(n_components=2)

In [10]:
# 로지스틱 분류 모델링 객체를 생성 
lr = LogisticRegression()

# TF-IDF 벡터를 입력하여 모델 학습 
lr.fit(review_tfid, labels)

LogisticRegression()

In [11]:
title_list = []
uid_list = []
url_list = []

text_list = []

for page in range(1,9):
    url = f'https://movie.naver.com/movie/bi/mi/review.naver?code=154667&page={page}'
    
    # requests.get
    response = requests.get(url)
    # BeautifulSoup 함수로, HTML 문서 구조를 파싱 
    soup = BeautifulSoup(response.content,'html.parser')

    # ul 태그의 class 속성값을 활용하여 리뷰 제목과 링크가 포함되어 있는 요소를 찾기
    review_list_tag = soup.find(name='ul', attrs={'class': 'rvw_list_area'})
    # review_list_tag 요소에 포함된 li 태그를 모두 찾기
    review_list_tags = review_list_tag.find_all(name='li')

    for li_tag in review_list_tags:
        # 리뷰 제목(rli.title) 데이터 추출
        review_title = li_tag.find_all('a')[0].get_text()
        title_list.append(review_title)
        # 사용자(rli.uid) 데이터 추출
        review_uid = li_tag.find_all('a')[1].get_text()
        uid_list.append(review_uid)
        # 각 리뷰 페이지로 연결할 nid 값 추출
        review_nid = re.findall('\d{7}', li_tag.find('a').get('onclick'))[0]
        # URL을 조합.  
        review_url = f"https://movie.naver.com/movie/bi/mi/reviewread.naver?nid={review_nid}&code=154667&order=#tab"
        url_list.append(review_url)

# url 반복하여 텍스트를 추출하고 리스트에 추가
for url in url_list:
    # 리뷰 상세페이지의 HTML 소스코드를 가져와서 파싱(parsing)
    resp_text = requests.get(url)
    soup_text = BeautifulSoup(resp_text.text, 'html.parser')

    # 리뷰 본문의 텍스트를 추출합니다. /  <div class="user_tx_area"> )
    review_text_tag = soup_text.find(name='div', attrs={'class':'user_tx_area'})

    # 텍스트 부분만 추출합니다.
    review_text = review_text_tag.get_text()
    text_list.append(review_text)
    
    # 대기 시간을 추가합니다. (서버에 과도한 호출이 되지 않도록 유의) 
    time.sleep(2)


In [12]:
text_list 

['\n포스터만 봐도 울컥 하네요.지금 우리 사회에는 맞벌이 부부로 인한, 혹은 이혼, 사별 등의 이유로조손가정이 늘고 있습니다.\xa0저출산 대책은 (물론 탁상공론에 어이없는 대책들 뿐일지라도) 끊임없이 논의 되지만,조손가정을 위한 지원 방안이 강구되지 않는것은, 우리사회가 역시 눈에 보이는 것에만신경쓰고 있다는 방증이 아닐까 싶습니다.\xa0이 영화를 통해 모든 조손 가정이 관심받고 지원받기를 바랍니다.\xa0세상의 모든 조손가정의 할머니 할아버님들 건강하시고손자 손녀들 행복하고 건강하기를 진심으로 기원해 봅니다.\xa0\n',
 '\n\r\n평점 9.5가 가뿐히 넘길래 믿었었는데저를 감정이 메마른사람이라고 욕해도 좋습니다50분이 넘도록 하품만하다가 곧 무슨일이있겠지 보다가도저히 지루함을 못참아서 그냥도중에 나왓습니다저로서는 네이버댓글평점알바조작 이라고밖에 생각을못하겠습니다사실확인여부도 안될뿐더러 돈이너무아까웠고 소비자 기만이고 댓글평점 조작은엄연한 소비자 기만이라고 생각합니다이순재 라는 배우의 연기는 모두 완벽했습니다.조연들의 연기도 인물구현, 감정 다 좋았습니다하지만 인물과 인물의 감정연결 스토리 등이 전혀 와닿지않고 전혀 이입이 되지않았습니다.\n',
 '\n정말 뜻밖에도 YES24 이벤트에 당첨되어영화 덕구 시사회에 다녀왔어요!처음 방문한 롯데시네마 수원에서 관람한영화 덕구 후기입니다 :)할배와 손자가 꼭 껴안고 있는포스터부터 가슴 찡한 눈물지뢰...ㅠㅠ아들이 죽고 며느리가 고향\xa0인도네시아로 돌아간 뒤,일곱 살 덕구(정지훈), 다섯 살 덕희(박지윤)와 함께시골마을에 사는 괴팍한 일흔 살 덕구 할배(이순재).곧 초등학교에 입학하는 덕구의 손을입학식 때 잡아주는 게 꿈입니다.하지만 암을 진단받은 덕구 할배는자신에게 남은 시간이 얼마 남지 않았음을 직감하고아이들을 위해 위탁 가정을 알아봅니다.다문화 가정의 애틋한 사랑을 그린 영화 \'덕구\'.91분이라는 짧은 러닝타임 때문에스토리 전개가 너무 급박하지 않을까 했는데,다큐멘터리 같은 드라마 장르의 영

In [13]:
# 딕셔너리 형식으로 항목별 리스트를 원소로 추가 
dict_data = {
    'title' : title_list,
    'user' : uid_list,
    'review' : text_list   
}

df = pd.DataFrame(dict_data)


In [14]:
# 변환 결과를 확인
df

,title,user,review
0,지금 우리사회 현실의 단편,apol****,"\n포스터만 봐도 울컥 하네요.지금 우리 사회에는 맞벌이 부부로 인한, 혹은 이혼,..."
1,이영화덕분에 댓글알바가 있다는걸 알았습니다.,hj93****,\n\r\n평점 9.5가 가뿐히 넘길래 믿었었는데저를 감정이 메마른사람이라고 욕해도...
2,영화 덕구 후기 (YES24 시사회),thdu****,\n정말 뜻밖에도 YES24 이벤트에 당첨되어영화 덕구 시사회에 다녀왔어요!처음 방...
3,감동영화 덕구 시사회후기 : 이순재 주연,mina****,"\n감동영화 덕구 시사회 후기 입니다.할아버지와 손자의 사랑, 가족애를 느낄 수 있..."
4,이제 마이묵었다 아이가,quie****,\n일구:덕구야 영화 잼있지? 덕구:재미는 없는데 너무감동적이야 이건 우리사회의 슬...
...,...,...,...
73,2018년 드라마 감동주는 영화 덕구 개봉 히트다 히트 가즈아,dldn****,\n\n안녕하세요 여러분 짬뽕입니다`~\n이번에 소개하는 것은\n영화\n드라마 감동...
74,깜짝초대 <덕구> 무대인사 상영회 140석 초대합니다.,movi****,"\n입소문 좋은 영화죠?게다가 무대인사, 일요일 <덕구> 상영회에 네영카 회원 14..."
75,덕구 영화시사회...이순재할아버지가 연기해 더더욱 빛나는~ 근데 슬펐다,love****,\n수원 롯데시네마는 처음인데 수원역에 위치하고 있어 접근성 좋아 이용하기 편하네요...
76,"덕구 Stand by me, 2017",skin****,"\n ""당신에게도 소중한 사람이 있습니까?""어린 손자와 살고 있는 일흔 살 덕구 할..."


In [15]:
df = df.iloc[:30]

In [16]:
df

,title,user,review
0,지금 우리사회 현실의 단편,apol****,"\n포스터만 봐도 울컥 하네요.지금 우리 사회에는 맞벌이 부부로 인한, 혹은 이혼,..."
1,이영화덕분에 댓글알바가 있다는걸 알았습니다.,hj93****,\n\r\n평점 9.5가 가뿐히 넘길래 믿었었는데저를 감정이 메마른사람이라고 욕해도...
2,영화 덕구 후기 (YES24 시사회),thdu****,\n정말 뜻밖에도 YES24 이벤트에 당첨되어영화 덕구 시사회에 다녀왔어요!처음 방...
3,감동영화 덕구 시사회후기 : 이순재 주연,mina****,"\n감동영화 덕구 시사회 후기 입니다.할아버지와 손자의 사랑, 가족애를 느낄 수 있..."
4,이제 마이묵었다 아이가,quie****,\n일구:덕구야 영화 잼있지? 덕구:재미는 없는데 너무감동적이야 이건 우리사회의 슬...
5,62년 연기인생 이순재 할아버지 영화 '덕구',rhdl****,\n62년 연기인생 이순재 할아버지 영화 '덕구'1935년 함경북도 회령 출신 어느...
6,정말정말 ..감동있는 영화입니다.,ekdu****,\n리뷰같은거...안쓰는데...이 영화는 리뷰를 쓰게 만듭니다. 이순재 할아버지의 ...
7,관객 가슴 미어지게 만들려고 작정한 영화,df34****,\r\n\t\t\t\t지루하진 않았음 하지만 막 와 개잼 개띵작 ~~~ 이런 느낌도...
8,<强 재미!> 마지막 울컥! 이순재&정지훈 최고의 연기와 감동! [배우중심 Char...,char****,\n덕구 개요 드라마감독 방수인출연 이순재 정지훈 장...
9,정말 감동적인 이야기,youn****,\n이번에 영화 덕구 시사회 다녀왔어요사실 별 기대하지 않는 작품이었어요~왜냐하면 ...


In [17]:
cleaned_tokens = []
for review in tqdm(df["review"]):
    # 한글을 제외하고 전부 제거
    text = re.sub(r"[^가-힣]", "", review)

    # 세글자 이상의 명사만을 추출
    tokens = okt.nouns(text)

    for word in tokens:
        if len(word) >= 2:
            cleaned_tokens.append(word)
        else:
            pass
    cleaned_review = " ".join(cleaned_tokens)


100%|██████████| 30/30 [06:55<00:00, 13.85s/it]


In [18]:
cleaned_review

'포스터 지금 우리 사회 맞벌이 부부 이혼 사별 유로 정이 저출산 대책 탁상공론 대책 논의 조손 가정 한지원 방안 이강 우리 사회 역시 신경 방증 이영화 통해 조손 정이 관심 지원 세상 조손 정의 할머니 아버님 손자 손녀 진심 기원 평점 감정 사람 해도 하품 만하 다가 무슨 일이 도저히 지루함 그냥 도중 왓습 로서 네이버 댓글 평점 알바 조작 생각 사실 확인 여부 뿐더러 소비자 기만 댓글 평점 조작 소비자 기만 생각 이순재 배우 연기 모두 조연 연기 인물 구현 감정 인물 인물 감정 연결 스토리 전혀 이입 정말 이벤트 당첨 화덕 시사회 처음 방문 롯데 시네마 수원 관람 한영 화덕 후기 할배 손자 포스터 가슴 눈물 지뢰 아들 며느리 고향 인도네시아 살덕 구정 지훈 살덕 박지윤 시골 마을 일흔 살덕구 할배 이순재 초등학교 입학 구의 입학식 진단 은덕 할배 자신 시간 얼마 남지 직감 아이 위해 위탁가정 다문화 정의 사랑 그린 영화 구분 러닝 타임 때문 스토리 전개 다큐멘터리 드라마 장르 영화 줄거리 중간 중간 날씨 연출 할배 구의 감정 선도 영화 구의 실제 주인공 인덕 가족 사진 마지막 감동 국민 할배 이순재 선생님 성질 시골 할배 연기 기대 이상 하이킥 제외 엄격 한역 자주 슬픔 가슴 누구 손자 사랑 모습 정말 연기 기도 영화 초반 장면 중반 할배 상태 손자 위해 결정 장면 계속 눈물 경쟁률 구역 아역배우 정지훈 드라마 도깨비 아버지 연기 번영 대사 전달 연기력 모습 할배 보고 정지훈 완벽 한덕 다문화 가정 사투리 설정 이순재 선생님 최고 의합 정말 아버지 손자 엄마 모습 모든 관객 한마음 고우리 맑은 이번 연기도전 대사 모습 눈물 계속 눈물샘 이마 감동 영화 덕구 시골 할아버지 손자 설정 가족 애가 부각 감동 더크 영화 진눈 지하철 살짝 민망 전체 작품 영화 덕구 년월 개봉 가족 보기 가슴 영화로 추천 요추 가원 오후 시작 시사회 콘텐츠 재생 계속 오류 상영 시작 문제 상영 시작 알았던 관객 무시 콘텐츠 재생 오류 사실 게다가 영화 시작 한지 난후 갑자기 영화 알

In [19]:
# TF-IDF 변환기에 데이터를 입력하여 변환
test_review_tfid = tfid.transform([cleaned_review])

In [20]:
# 로지스틱 회귀 모델을 활용하여 분류 예측
test_pred = lr.predict(test_review_tfid)
print("분석 결과 {}적인 리뷰로 예측됩니다. ".format("긍정" if test_pred > 0 else "부정"))

분석 결과 긍정적인 리뷰로 예측됩니다. 
